### Data_Preperation

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import faiss
from langchain_community.vectorstores import FAISS
import os
from openai import AzureOpenAI, OpenAI
import pyarabic.araby as araby
import nltk
import tiktoken
import time
import re
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions

In [6]:
def english_to_arabic_numbers(text):
    number_mapping = str.maketrans('0123456789', '٠١٢٣٤٥٦٧٨٩')
    return text.translate(number_mapping)

In [7]:
file_path = 'Dostoor_Egy_Structured.xlsx'

In [8]:
df = pd.read_excel(file_path)
df

,Bab,Fasl,Madda,Text,Bab_name,fasl_name
0,1,1,1,مادة (١)\nجمهورية مصر العربية دولة ذات سيادة، ...,الباب الأول: الدولة,NaN
1,1,1,2,مادة (٢)\nالإسلام دين الدولة، واللغة العربية ل...,الباب الأول: الدولة,NaN
2,1,1,3,مادة (٣)\nمبادئ شرائع المصريين من المسيحيين وا...,الباب الأول: الدولة,NaN
3,1,1,4,مادة (٤)\nالسيادة للشعب وحده، يمارسها ويحميها،...,الباب الأول: الدولة,NaN
4,1,1,5,مادة (٥)\nيقوم النظام السياسي على أساس التعددي...,الباب الأول: الدولة,NaN
...,...,...,...,...,...,...
253,7,1,251,مادة (٢٥١)\nيشترط فيمن يترشح لعضوية مجلس الشيو...,الباب السابع: مجلس الشيوخ,NaN
254,7,1,252,مادة (٢٥٢)\nلا يجوز الجمع بين عضوية مجلس الشيو...,الباب السابع: مجلس الشيوخ,NaN
255,7,1,253,مادة (٢٥٣)\nرئيس مجلس الوزراء ونوابه والوزراء ...,الباب السابع: مجلس الشيوخ,NaN
256,7,1,254,مادة (٢٥٤)\nتسرى فى شأن مجلس الشيوخ الأحكام ال...,الباب السابع: مجلس الشيوخ,NaN


In [9]:
# filling NaN values in fasl_name with empty strings to avoid concatenation issues
df['fasl_name'] = df['fasl_name'].fillna('')
df['Bab_name'] = df['Bab_name'].fillna('')

df['Text'] = df['Bab_name'] + '\n' + df['fasl_name'] + '\n' + df['Text']

In [10]:
print(df['Text'][120])

 الباب الخامس: نظام الحكم
الفصل الأول
السلطة التشريعية (مجلس النواب)
مادة (١٢١)
لا يكون انعقاد المجلس صحيحًا، ولا تتخذ قراراته، إلا بحضور أغلبية أعضائه.
وفى غير الأحوال المشترط فيها أغلبية خاصة، تصدر القرارات بالأغلبية المطلقة للحاضرين، وعند تساوى الآراء، يعتبر الأمر الذى جرت المداولة فى شأنه مرفوضًا.
وتصدر الموافقة على القوانين بالأغلبية المطلقة للحاضرين، وبما لا يقل عن ثلث عدد أعضاء المجلس.
كما تصدر القوانين المكملة للدستور بموافقة ثلثى عدد أعضاء المجلس. وتعد القوانين المنظمة للانتخابات الرئاسية، والنيابية، والمحلية، والأحزاب السياسية، والسلطة القضائية، والمتعلقة بالجهات والهيئات القضائية، والمنظمة للحقوق والحريات الواردة فى الدستور، مكملة له.


In [11]:
output_csv_path = 'Final_Dostoor_Structured.csv'
df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [12]:
text_list = df['Text'].tolist()
text_list

['الباب الأول: الدولة\n\nمادة (١)\nجمهورية مصر العربية دولة ذات سيادة، موحدة لا تقبل التجزئة، ولا ينزل عن شئ منها، نظامها جمهورى ديمقراطى، يقوم على أساس المواطنة وسيادة القانون.\nالشعب المصرى جزء من الأمة العربية يعمل على تكاملها ووحدتها، ومصر جزء من العالم الإسلامى، تنتمى إلى القارة الأفريقية، وتعتز بامتدادها الآسيوى، وتسهم فى بناء الحضارة الإنسانية.',
 'الباب الأول: الدولة\n\nمادة (٢)\nالإسلام دين الدولة، واللغة العربية لغتها الرسمية، ومبادئ الشريعة الإسلامية المصدر الرئيسى للتشريع.',
 'الباب الأول: الدولة\n\nمادة (٣)\nمبادئ شرائع المصريين من المسيحيين واليهود المصدر الرئيسى للتشريعات المنظِّمة لأحوالهم الشخصية، وشئونهم الدينية، واختيار قياداتهم الروحية.',
 'الباب الأول: الدولة\n\nمادة (٤)\nالسيادة للشعب وحده، يمارسها ويحميها، وهو مصدر السلطات، ويصون وحدته الوطنية التى تقوم على مبادئ المساواة والعدل وتكافؤ الفرص بين جميع المواطنين، وذلك على الوجه المبين فى الدستور.',
 'الباب الأول: الدولة\n\nمادة (٥)\nيقوم النظام السياسي على أساس التعددية السياسية والحزبية، والتداول السلمى للسلطة، وا

### Embedding and RAG

In [13]:
question = 'بماذا يحاكم المسيحيين بمصر؟'

In [14]:
model = SentenceTransformer("intfloat/multilingual-e5-base")

In [15]:
# paragraph_embedding = model.encode(text_list)

In [16]:
question_embedding = model.encode([question])

In [17]:
# paragraph_embedding.shape

In [18]:
# d = question_embedding.shape[1]
# index = faiss.IndexFlatIP(d)
# index.add(paragraph_embedding)

# D, I = index.search(question_embedding, k = 5)  # search
# I = I[0]

In [19]:
# I

In [20]:
# context = [text_list[k] for k in I]
# context

In [21]:
# results['documents'][0]

## Chat

In [22]:
## Keys ##
OPENAI_API_TYPE = os.getenv("OPENAI_API_TYPE")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")
DEPLOYMENT_URL =  os.getenv("DEPLOYMENT_URL")
# MAILGUN_API_KEY = os.getenv("MAILGUN_API_KEY")

def ConnectToAzure():
    """
    desc:
        Function connects to langchain AzureOpenAI using keys in .env file
    params:
        None
    return: 
        model: model of llm
    """
    model = AzureChatOpenAI(
        openai_api_base=OPENAI_API_BASE,
        openai_api_version=OPENAI_API_VERSION,
        azure_deployment=DEPLOYMENT_NAME,
        openai_api_key=OPENAI_API_KEY,
        openai_api_type=OPENAI_API_TYPE,
    )
    return model

In [23]:
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="intfloat/multilingual-e5-base")

In [24]:
# chroma_client = chromadb.PersistentClient(path="Dostor_embedding_VDB")
# db = chroma_client.create_collection(name='Dostor_embedding_VDB', embedding_function=embedding_model)

# for i, d in enumerate(text_list):
#     db.add(
#       documents=d,
#       ids=str(i)
#     )

Get VDB

In [25]:
chroma_client = chromadb.PersistentClient(path="Dostor_embedding_VDB")
db = chroma_client.get_collection(name="Dostor_embedding_VDB", embedding_function=embedding_model)

In [44]:
question = "قارن بين المادة 170 و المادة 171"
question = english_to_arabic_numbers(question)

In [45]:
question

'قارن بين المادة ١٧٠ و المادة ١٧١'

In [27]:
results = db.query(query_texts=question, n_results=15, include=['distances', 'documents']) 
context = results['documents'][0]

In [43]:
context[14]

'الباب الرابع: سيادة القانون\n\nمادة (٩٥)\nالعقوبة شخصية، ولا جريمة ولا عقوبة إلا بناء على قانون، ولا توقع عقوبة إلا بحكم قضائى، ولا عقاب إلا على الأفعال اللاحقة لتاريخ نفاذ القانون.'

In [29]:
def Chat():

    _DEFAULT_TEMPLATE = """
    you are an Arabic Lawyer who have knowledge in the constitution of Egypt, 
    Given the following context and question in Arabic find a precise Arabic answer
    . Your answer should be ONLY in Arabic and always mention the section that you get your answer from .\
    
    context: {context} \
    
    Current conversation:
    New human question: {input}
    Response:"""

    prompt = PromptTemplate(
        input_variables=["input","context"], template=_DEFAULT_TEMPLATE
    )

    conversation = LLMChain(
        llm=ConnectToAzure(),
        prompt=prompt,
        verbose=True,
    )
    return conversation

In [51]:
Chat = Chat()
answer = Chat.predict(input = question,context = context)

C:\Users\Tony\AppData\Roaming\Python\Python311\site-packages\langchain_community\chat_models\azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://eslsca-openai.openai.azure.com/ to https://eslsca-openai.openai.azure.com/openai.
  warnings.warn(
C:\Users\Tony\AppData\Roaming\Python\Python311\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\Tony\AppData\Roaming\Python\Python311\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of th



> Entering new LLMChain chain...
Prompt after formatting:

    you are an Arabic Lawyer who have knowledge in the constitution of Egypt, 
    Given the following context and question in Arabic find a precise Arabic answer
    . Your answer should be ONLY in Arabic and always mention the section that you get your answer from .    
    context: ['الباب السابع: مجلس الشيوخ\n\nمادة (٢٥٢)\nلا يجوز الجمع بين عضوية مجلس الشيوخ وعضوية مجلس النواب.', 'الباب السابع: مجلس الشيوخ\n\nمادة (٢٥٤)\nتسرى فى شأن مجلس الشيوخ الأحكام الواردة بالدستور فى المواد ١٠٣، ١٠٤، ١٠٥،، ١٠٧، ١٠٨، ١٠٩، ١١٠، ١١١، ١١٢، ١١٣، ١١٤، ١١٥، ١١٦، ١١٧، ١١٨، ١١٩، ١٢٠، (١٢١/ فقرة ١، ٢)، ١٣٢، ١٣٣، ١٣٦، ١٣٧. وذلك فيما لا يتعارض مع الأحكام الواردة فى هذا الباب، وعلى أن يباشر الاختصاصات المقررة فى المواد المذكورة مجلس الشيوخ ورئيسه.', 'الباب السادس: الأحكام العامة والانتقالية\n\nمادة (٢٤٣) (١)\nتعمل الدولة على تمثيل العمال والفلاحين تمثيلاً ملائماً فى مجلس النواب، وذلك على النحو الذى يُحدده القانون.\n\n____________________\n\n(١)، 

In [52]:
print(answer)

المادة ١١٠ تتناول إسقاط العضوية في مجلس النواب وتنص على أنه يجوز إسقاط عضوية العضو إذا فقد الثقة والاعتبار، أو فقد أحد شروط العضوية التي انتُخب على أساسها، أو أخل بواجبات عضويته. ويجب أن يصدر قرار إسقاط العضوية من المجلس بأغلبية ثلثي أعضائه.

أما المادة ١١٥، فتتناول مواعيد انعقاد جلسات مجلس النواب، وتنص على أن رئيس الجمهورية يدعو المجلس للانعقاد للدور العادي السنوي قبل يوم الخميس الأول من شهر أكتوبر، فإذا لم تتم الدعوة يجتمع المجلس بحكم الدستور في اليوم المذكور. ويستمر دور الانعقاد العادي لمدة لا تقل عن تسعة أشهر. ويفض رئيس الجمهورية دور الانعقاد بعد موافقة المجلس على ذلك، ولا يجوز ذلك قبل اعتماد الموازنة العامة للدولة.

المواد من دستور مصر:
- المادة ١١٠: الباب الخامس: نظام الحكم، الفصل الأول: السلطة التشريعية (مجلس النواب)
- المادة ١١٥: الباب الخامس: نظام الحكم، الفصل الأول: السلطة التشريعية (مجلس النواب)
